In [51]:
from DB import DataFrameHandler
from DB import AIModelHandler
import pandas as pd
import pickle, os

pd.options.mode.chained_assignment = None

def preprocessing(ohlcv, fundamental, dateDF):
    tmp_ohlcv = ohlcv.drop(columns=['AMOUNT'])
    try:
        tmp_fundamental = fundamental[["ISIN","WORK_DT", 'PBR','PER']]
    except:
        tmp_fundamental = pd.DataFrame([])
    total = pd.merge(tmp_ohlcv, tmp_fundamental, how='outer').dropna(axis=1)
    
    total.drop(columns=["ISIN"], inplace=True)
    total['WORK_DT'] = dateDF['DATE_LABEL']

    return total

def callTargetDataFrame(isin, date):
    if DataFrameHandler.findItems('stock', columns='count(*)', condition=f"ISIN='{isin}'")['COUNT(*)'].to_list()[0] != 1:
        raise Exception("ISIN NOT EXIST")
    dateDF = DataFrameHandler.findItems('WORK_DATE', condition=f"WORK_DT >= to_date('{date}', 'YYYY/MM/DD')")
    ohlcv = DataFrameHandler.findItems('ohlcv', condition=f"ISIN='{isin}' and WORK_DT >= to_date('{date}', 'YYYY/MM/DD')", orderBy='WORK_DT')
    fundamental = DataFrameHandler.findItems('fundamental', condition=f"ISIN='{isin}' and WORK_DT >= to_date('{date}', 'YYYY/MM/DD')", orderBy='WORK_DT')
    result = preprocessing(ohlcv, fundamental, dateDF)
    return {"result" : result, "date" : dateDF}

def callAIModels(isin):
    if DataFrameHandler.findItems('stock', columns='count(*)', condition=f"ISIN='{isin}'")['COUNT(*)'].to_list()[0] != 1:
        raise Exception("ISIN NOT EXIST")
    mainDirectory = os.path.join('PredictModel', 'models')
    subDirectory = os.path.join(mainDirectory, isin)
    fileDirectory = os.path.join(subDirectory, 'lastest.sav')
    with open(fileDirectory, 'rb') as file:
        models = file.read()
    models = pickle.loads(models)
    return models

def predict(isin, date):
    datas = callTargetDataFrame(isin, date)
    dates = datas['date']
    dataFrame = datas['result']
    models = callAIModels(isin)
    priceModel = models['priceModel']
    updownModel = models['updownModel']
    pricePredict = priceModel.predict(dataFrame)
    updownPredict = updownModel.predict(dataFrame)
    result = pd.DataFrame({"pricePredict" : pricePredict, "updownPredict" : updownPredict})
    dataFrame['WORK_DT'] = dates['WORK_DT']
    dataFrame['ISIN'] = isin
    result = pd.concat([dataFrame, result], axis=1)
    return result


def calcPriceScore(x, y, model):
    answer = (y > x['CLOSE']).reset_index(drop=True)
    prediction = model.predict(x) > x['CLOSE'].reset_index(drop=True)
    predictResult = (answer == prediction) * 1
    return round(sum(predictResult)/len(predictResult),2)

def calcUpdownScore(x, y, model):
    return round(model.score(x, y),2)

In [52]:
df = callTargetDataFrame('005930', '2023-09-01')
df

{'result':     WORK_DT   OPEN   HIGH    LOW  CLOSE    VOLUME  UPDOWN   PBR   PER
 0      2628  66800  71000  66700  71000  29738235    6.13  1.23  8.81
 1      2629  72900  72900  70700  71200  26286495    0.28  1.23  8.84
 2      2630  70900  71500  70200  70700  12330239   -0.70  1.22  8.77
 3      2631  70700  70800  69700  70000  11414620   -0.99  1.21  8.69
 4      2632  70000  70600  69600  70400  13741241    0.57  1.22  8.74
 5      2633  70200  70300  69600  70300  10688118   -0.14  1.22  8.73
 6      2634  70400  70800  70000  70800  11785462    0.71  1.22  8.79
 7      2635  70800  71000  70400  70500  11688599   -0.42  1.22  8.75
 8      2636  71100  71600  70300  70900  15955797    0.57  1.23  8.80
 9      2637  71200  71800  70800  71700  21041407    1.13  1.24  8.90
 10     2638  71700  72300  71400  72000  17823512    0.42  1.25  8.94
 11     2639  71300  71700  70200  70200  16040727   -2.50  1.21  8.71
 12     2640  70400  70800  69600  69800  11820188   -0.57  1.21  8

In [53]:
models = callAIModels('005930')
models

{'priceModel': LogisticRegression(solver='saga'),
 'updownModel': LogisticRegression(),
 'priceModelScore': 0.53,
 'updownModelScore': 0.52}

In [59]:
prediction = predict('005930', '2023-09-01')
prediction

WORK_DT   OPEN   HIGH    LOW  CLOSE    VOLUME  UPDOWN   PBR   PER   
0  2023-09-01  66800  71000  66700  71000  29738235    6.13  1.23  8.81  \
1  2023-09-04  72900  72900  70700  71200  26286495    0.28  1.23  8.84   
2  2023-09-05  70900  71500  70200  70700  12330239   -0.70  1.22  8.77   
3  2023-09-06  70700  70800  69700  70000  11414620   -0.99  1.21  8.69   
4  2023-09-07  70000  70600  69600  70400  13741241    0.57  1.22  8.74   
5  2023-09-08  70200  70300  69600  70300  10688118   -0.14  1.22  8.73   
6  2023-09-11  70400  70800  70000  70800  11785462    0.71  1.22  8.79   
7  2023-09-12  70800  71000  70400  70500  11688599   -0.42  1.22  8.75   
8  2023-09-13  71100  71600  70300  70900  15955797    0.57  1.23  8.80   
9  2023-09-14  71200  71800  70800  71700  21041407    1.13  1.24  8.90   
10 2023-09-15  71700  72300  71400  72000  17823512    0.42  1.25  8.94   
11 2023-09-18  71300  71700  70200  70200  16040727   -2.50  1.21  8.71   
12 2023-09-19  70400  70800  69600  69800  11820188   -0.57  1.21  8.66   
13 2023-09-20  70000  70300  69500  69600  10873015   -0.29  1.20  8.64   
14 2023-09-21  69200  69800  68800  68900  10796336   -1.01  1.19  8.55   
15 2023-09-22  68300  68900  68300  68800   9897840   -0.15  1.19  8.54   
16 2023-09-25  68500  69700  68500  69400  13582516    0.87  1.20  8.61   

      ISIN  pricePredict  updownPredict  
0   005930       59000.0            0.0  
1   005930       59000.0            0.0  
2   005930       59000.0            0.0  
3   005930       59000.0            0.0  
4   005930       59000.0            0.0  
5   005930       59000.0            0.0  
6   005930       59000.0            0.0  
7   005930       59000.0            0.0  
8   005930       59000.0            0.0  
9   005930       59000.0            0.0  
10  005930       59000.0            0.0  
11  005930       59000.0            0.0  
12  005930       59000.0            0.0  
13  005930       59000.0            0.0  
14  005930       59000.0            0.0  
15  005930       59000.0            0.0  
16  005930       59000.0            0.0

In [108]:
stockMapper = {}
newStock =[]
for i, stock in enumerate(ohlcv):
    isin = stock["ISIN"][0]
    if isin in ['004565' ,'353060', '010145', '000995', '010050', '000547', '001745']:
        continue

    stockMapper[isin] = preprocessing(stock, fundamental[i])
    newModel = makeAiModel(stockMapper[isin]["train"], 29)
    createNewModels(isin,newModel,"Ver00001")

0번째 : 000020
1번째 : 000040
2번째 : 000050
3번째 : 000070
4번째 : 000075
5번째 : 000080
6번째 : 000087
7번째 : 000100
8번째 : 000105
9번째 : 000120
10번째 : 000140
11번째 : 000145
12번째 : 000150
13번째 : 000155
14번째 : 000157
15번째 : 000180
16번째 : 000210
17번째 : 000215
18번째 : 000220
19번째 : 000225
20번째 : 000227
21번째 : 000230
22번째 : 000240
23번째 : 000270
24번째 : 000300
25번째 : 000320
26번째 : 000325
27번째 : 000370
28번째 : 000390
29번째 : 000400
30번째 : 000430
31번째 : 000480
32번째 : 000490
33번째 : 000500
34번째 : 000520
35번째 : 000540
36번째 : 000545
38번째 : 000590
39번째 : 000640
40번째 : 000650
41번째 : 000660
42번째 : 000670
43번째 : 000680
44번째 : 000700
45번째 : 000720
46번째 : 000725
47번째 : 000760
48번째 : 000810
49번째 : 000815
50번째 : 000850
51번째 : 000860
52번째 : 000880
53번째 : 000885
54번째 : 00088K
55번째 : 000890
56번째 : 000910
57번째 : 000950
58번째 : 000970
59번째 : 000990
61번째 : 001020
62번째 : 001040
63번째 : 001045
64번째 : 00104K
65번째 : 001060
66번째 : 001065
67번째 : 001067
68번째 : 001070
69번째 : 001080
70번째 : 001120
71번째 : 001130
72번째 : 001140
73번째 : 001200
74

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
